In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd Plant

/content/drive/MyDrive/Colab Notebooks/Plant


In [ ]:
!mkdir Plant


In [ ]:
ls

Palnt_Patho_2021.ipynb  Plant/


In [ ]:
cd Plant

/content/drive/My Drive/Colab Notebooks/Plant


In [ ]:
!mkdir train_images

In [ ]:
ls

sample_submission.csv  train_comp.csv  train_images_comp.rar
test_images/           train_images/


In [ ]:
!unrar x train_images_comp.rar train_images 

In [ ]:
ls -1 /content/drive/MyDrive/Colab\ Notebooks/Plant/train_images/ | wc -l

18632


In [ ]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Plant'

In [ ]:
ls

sample_submission.csv  train_comp.csv  train_images_comp.rar
test_images/           train_images/


In [ ]:
ls -1 /content/drive/MyDrive/Colab\ Notebooks/Plant/train_images/ | wc -l

18632


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
import cv2
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow import keras

from keras import Sequential
from tensorflow.keras.applications import InceptionResNetV2, Xception, MobileNetV2
from tensorflow.keras.layers import Dense


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from time import time


from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# load dataset

# sur colab
# TRAIN_IMAGES_PATH = "F:/Projet_computer_vision/plant-pathology-2021-fgvc8/train_images/"

# # en local
# TRAIN_IMAGES_PATH = "F:/Projet_computer_vision/plant-pathology-2021-fgvc8/compressed/train_images/"
# TEST_IMAGES_PATH = "F:/Projet_computer_vision/plant-pathology-2021-fgvc8/test_images/"
# sur colab
TRAIN_IMAGES_PATH = "drive/MyDrive/Colab Notebooks/Plant/train_images/"
TEST_IMAGES_PATH = "drive/MyDrive/Colab Notebooks/Plant/test_images/"

# en local
# TRAIN_PATH = "F:/Projet_computer_vision/plant-pathology-2021-fgvc8/compressed/train_comp.csv"
# SUB_PATH = "F:/Projet_computer_vision/plant-pathology-2021-fgvc8/sample_submission.csv"

# sur colab
TRAIN_PATH = "drive/MyDrive/Colab Notebooks/Plant/train_comp.csv"
SUB_PATH = "drive/MyDrive/Colab Notebooks/Plant/sample_submission.csv"

train_data = pd.read_csv(TRAIN_PATH)
print("train_data= " , train_data[:5],"\n")
test_data = pd.read_csv(SUB_PATH)
print("test_data= ", test_data,"\n")
# sub = pd.read_csv(SUB_PATH)

# bon 3
classes=set()
for v in train_data["labels"].values:
    classes.update(set(v.split()))
print("classes= ", classes,"\n")

#On peut associer les images des feuilles à 5 classes possibles:
#'complex', 'frog_eye_leaf_spot', 'powdery_mildew', 'rust', 'scab'
# healthy , quand aucune de ces classes n'est là.

train_data['scab']=train_data['labels'].apply(lambda x: 1 if 'scab' in x else 0) 
train_data['rust']=train_data['labels'].apply(lambda x: 1 if 'rust' in x else 0) 
train_data['powdery_mildew']=train_data['labels'].apply(lambda x: 1 if 'powdery_mildew' in x else 0) 
train_data['frog_eye_leaf_spot']=train_data['labels'].apply(lambda x: 1 if 'frog_eye_leaf_spot' in x else 0) 
train_data['complex']=train_data['labels'].apply(lambda x: 1 if 'complex' in x else 0) 
# train_data
print("train_data ave 5 colonnes de labels = ")
print(train_data[:3],"\n")

labels_titles=["scab","rust","powdery_mildew","frog_eye_leaf_spot","complex"]

# sur colab
def add_path_compressed_train(file):
    return TRAIN_IMAGES_PATH  + file

def add_path_test(file):
    return TEST_IMAGES_PATH + file

train_files= train_data.image.map(add_path_compressed_train).values
print("train_files= \n", train_files[:3],"\n")

test_files= test_data.image.map(add_path_test).values
print("test_files= \n", test_files,"\n")

labels = np.float32(train_data.loc[:, 'scab':'complex'].values)
print("labels= \n ", labels[:3],"\n")

print("shapes= ", train_files.shape, labels.shape,"\n")

train_data=                               image                           labels
0  Compressed_800113bb65efe69e.jpg                          healthy
1  Compressed_8002cb321f8bfcdf.jpg  scab frog_eye_leaf_spot complex
2  Compressed_80070f7fb5e2ccaa.jpg                             scab
3  Compressed_80077517781fb94f.jpg                             scab
4  Compressed_800cbf0ff87721f8.jpg                          complex 

test_data=                    image   labels
0  85f8cb619c66b863.jpg  healthy
1  ad8770db05586b59.jpg  healthy
2  c7b03e718489f3ca.jpg  healthy 

classes=  {'rust', 'healthy', 'powdery_mildew', 'complex', 'frog_eye_leaf_spot', 'scab'} 

train_data ave 5 colonnes de labels = 
                             image  ... complex
0  Compressed_800113bb65efe69e.jpg  ...       0
1  Compressed_8002cb321f8bfcdf.jpg  ...       1
2  Compressed_80070f7fb5e2ccaa.jpg  ...       0

[3 rows x 7 columns] 

train_files= 
 ['drive/MyDrive/Colab Notebooks/Plant/train_images/Compressed_800113bb

In [ ]:
def imagedatagen():

        train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest")
        # validation_split=0.20)

        valid_test_datagen = ImageDataGenerator(rescale=1 / 255.0)
        
        test_datagen = ImageDataGenerator(rescale=1 / 255.0)
        
        
        
        return(train_datagen, valid_test_datagen, test_datagen)
    

def make_train_valid_test_generators(train_datagen,valid_test_datagen, test_datagen, train_data, valid_data, test_data, batch_size):
   
    
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory=TRAIN_IMAGES_PATH,
        x_col="image",
        y_col=labels_titles,
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode="raw",
 #       subset='training',
        shuffle=True,
        seed=42
    )
    valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_data,
        directory=TRAIN_IMAGES_PATH,
        x_col="image",
        y_col=labels_titles,
        target_size=(100, 100),
        batch_size=batch_size,
        class_mode="raw",
#        subset='validation',
        shuffle=True,
        seed=42
    )
    
        
    valid_test_generator = valid_test_datagen.flow_from_dataframe(
        dataframe=valid_data,
        directory=TRAIN_IMAGES_PATH,
        x_col="image",        
        target_size=(100, 100),
        batch_size=1,
        class_mode=None,
        shuffle=False,
    )   

    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_data,
        directory=TEST_IMAGES_PATH,
        x_col="image",
        target_size=(100, 100),
        batch_size=1,
        class_mode=None,
        shuffle=False,
    )

    

    return train_generator,valid_generator,valid_test_generator, test_generator


        

def get_model(image_shape):
    # InRes3
    #Adding the final layers to the above base models where the actual classification is done in the dense layers
    InResNetV2= InceptionResNetV2(include_top=False, weights="imagenet", \
        input_tensor=None, input_shape=image_shape, pooling='avg') 
    model_InResNet2 = Sequential()
    model_InResNet2.add(InResNetV2)
    model_InResNet2.add(Dense(5, activation=('sigmoid')))

    model_InResNet2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model_InResNet2.summary()
    return model_InResNet2


def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    # lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn   


def train(model, train_generator, valid_generator,epochs,  batch_size, filenum):
         
    
    # learning rate
    
    lrfn = build_lrfn()

    STEPS_PER_EPOCH = train_generator.n // batch_size
    VALIDATION_STEPS = valid_generator.n// batch_size
    # STEPS_PER_EPOCH = 10
    # VALIDATION_STEPS=4
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

    # Early Stopping
    early_stop= EarlyStopping(monitor='val_accuracy',patience=5, mode='max' )

    # save the weights given the best val_loss
    filepath='drive/MyDrive/Colab Notebooks/Plant/IncRestV2-best-gen-062021-'+str(filenum)+'.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [lr_schedule,early_stop, checkpoint ]
    # callbacks_list = [lr_schedule ]
    #Network=model.fit(train_dataset, steps_per_epoch=STEPS_PER_EPOCH, verbose=1, \
    #                            validation_data=valid_dataset,callbacks=callbacks_list,epochs=epochs)

    Network =model.fit(train_generator, validation_data = valid_generator, 
                             epochs =epochs,steps_per_epoch=STEPS_PER_EPOCH,
                       validation_steps = VALIDATION_STEPS,callbacks=callbacks_list)
    
#     Network =model_InResNet2.fit(train_generator, validation_data = valid_generator, 
#                               epochs =epochs,steps_per_epoch=STEPS_PER_EPOCH,
#                         validation_steps = VALIDATION_STEPS)
    
    
    return Network
    
    
def validate(model, validData, valid_test_generator,n): 
   
   
    valid_true_labels=np.float32(validData.loc[:, 'scab':'complex'].values)
    
    model = load_model("drive/MyDrive/Colab Notebooks/Plant/IncRestV2-best-gen-062021-"+str(n)+".hdf5")
    valid_pred = model.predict(valid_test_generator)
       
    
    # c'est exactement : accuracy_score(valid_true_labels, np.round(valid_pred)) (from sklearn.metrics import accuracy_score)
    accuracy= np.mean(np.all(valid_true_labels == np.round(valid_pred), axis=1))
    
    return accuracy

# une autre manière pour calculer la validation accuracy qui donne le même résultat.
def validate1(model, valid_dataset,len_valid): 
   
    valid_true_labels=np.float32(validData.loc[:, 'scab':'complex'].values)
    valid_pred = model.predict(valid_test_generator)
    
    accuracy_score(valid_true_labels, np.round(valid_pred))
    
    return accuracy

# fonction qui trace les 2 courbes loss et validation loss ainsi que l'accuracy et la validation accuracy
def loss_accuracy_curves(Network) :
    plt.subplot(121)
    plt.plot(Network.history['loss'],label='Train')
    plt.plot(Network.history['val_loss'],label='Validation')
    plt.ylabel('Loss',fontsize=20)
    plt.xlabel('epoch')
    plt.legend(['Training', 'Validation'], loc='best')  # upper

    plt.subplot(122)
    plt.plot(Network.history['accuracy'],label='Train')
    plt.plot(Network.history['val_accuracy'],label='Validation')
    plt.ylabel('Accuracy',fontsize=20)
    plt.xlabel('epoch')
    plt.legend(['Training', 'Validation'], loc='best')  # upper
    plt.tight_layout()

    plt.show()
    
    


In [ ]:
image_size=(100, 100)
AUTO = tf.data.experimental.AUTOTUNE
batch_size =16 # 16

X = train_data   # fichiers des images des feuilles (18 632)

cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1)

debut= time()

def evaluate_model_plant_patho(X):
    
 n=1
 list_val_accuracy=[]

 for train_ix, test_ix in cv.split(X):

    print("Split des données entre train set et validation set en utlisant la 3-fold cross validation ")

    trainData = X.iloc[train_ix]
    validData = X.iloc[test_ix]



    print(trainData.shape)  # 12 421 ou 12 422
    print(validData.shape) # 6 211 ou 6 210



    print("\nCréation des ImageDataGenerator pour le train, la validation à tester et pour le test set")    
    train_datagen, valid_test_datagen, test_datagen = imagedatagen() 

    print("\nCréation des train, validation et test generators ")

    train_generator,valid_generator,valid_test_generator, test_generator= make_train_valid_test_generators(train_datagen,\
                                     valid_test_datagen,test_datagen, trainData, validData, test_data, batch_size)

   

    print("\nCréation du modèle à partir d'une Keras application (deep learning pre-trained models)")
    print("https://keras.io/api/applications/ ")

    image_shape = image_size + (3,) 
    model= get_model(image_shape)

    print("\ntraining")
    # training du modèle et sauvegarde du meilleur modèle (max val_accuracy) rencontré lors des epochs (sauvegarde des poids)
    # train(model, train_dataset, valid_dataset, epochs=5, len_train=len_train, batch_size=16, filenum=n)
    Network= train(model, train_generator, valid_generator,epochs=15 ,  batch_size=batch_size, filenum=n)

   
    # break
    # faire une prediction du validation set à partir du modèle et la comparer aux vrais labels
    # ça équivaut à un accuracy_score de sklearn.
    # attention : c'est pas équivalent au model.evaluate(valid_dataset) de keras, ça ne donne pas les mêmes résultats car 
    # il s'agit d'une classification Multi-Class et multi-label.

    print("\n validation :prend du temps, car il faut faire une prévision pour tout le validation set: 6 211 images, le tiers de tout le dataset")
    val_accuracy = validate(model, validData, valid_test_generator,n) 
    
    
    # courbes de loss / validation loss et courbe d'accuracy / validation accuracy
    loss_accuracy_curves(Network) 
    
    # Sauvegarde de la val_accuracy dans uen liste pour un calcul de moyenne et d'écart-type
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>> validation_accuracy %.3f' % val_accuracy)
    list_val_accuracy.append(val_accuracy)
    
    print("\n >>>>>>>>>>>>>>>>>>>>>>>FIN Training et Cross validation :  ",n)
    print("\n\n")
   
    break;   
 
 return list_val_accuracy

list_val_accuracy= evaluate_model_plant_patho(X)
duree_totale = time()- debut
print("Durée des 3 trainings basés sur le 3-fold cross-validation  (en secondes) =", duree_totale)
print('\nValidation Accuracy: Moyenne %.3f Ecart-type(%.3f)' % (mean(list_val_accuracy), std(list_val_accuracy)))   


Split des données entre train set et validation set en utlisant la 3-fold cross validation 
(12421, 7)
(6211, 7)

Création des ImageDataGenerator pour le train, la validation à tester et pour le test set

Création des train, validation et test generators 
Found 12421 validated image filenames.
Found 6211 validated image filenames.
Found 6211 validated image filenames.
Found 3 validated image filenames.

Création du modèle à partir d'une Keras application (deep learning pre-trained models)
https://keras.io/api/applications/ 
219062272/219055592 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dense (Dense)                (None, 5)                 7685      
Total params: 54,344,421
Trainable params: 54,283,877